In [39]:
import pickle
import random

import torch
import torch.nn as nn
import torch.utils.data as data

from tg_GNNcVAE import GraphcVAE
from tg_Preprocessing import load_tg_data
from tg_config import tg_clf_config

In [6]:
epochs = tg_clf_config['epochs']
learning_rate = tg_clf_config['learning_rate']
CUDA = tg_clf_config['CUDA']
log_step = tg_clf_config['log_step']
save_step = tg_clf_config['save_step']
TRAIN = tg_clf_config['TRAIN']
plot_mAP = tg_clf_config['plot_mAP']
probability_threshold = tg_clf_config['probability_threshold']
model_path = tg_clf_config['model_path']
batch_size = tg_clf_config['batch_size']
n_encoding_feature = tg_clf_config['n_encoding_feature']
in_node = tg_clf_config['in_node']
data_size = tg_clf_config['tg_clf_config']

In [7]:
train_set:list = []
val_set:list = []
test_set:list = []
for i in range(data_size[0],data_size[1]):
    tmp_train_set, tmp_val_set, tmp_test_set = load_tg_data(num=i)
    train_set += tmp_train_set
    val_set += tmp_val_set
    test_set += tmp_test_set

In [8]:
GNN_cVAE = GraphcVAE(
     en_graph_in_channels=17338, en_graph_hidden_channels=256, en_graph_num_layers=1, en_graph_out_channels=128,
     hidden_channels=128, num_layers=1, z_dim=32,
     de_graph_in_channels=17338, de_graph_hidden_channels=256, de_graph_num_layers=1, de_graph_out_channels=256,
     num_z_layers=2,
     clf_num_layers=3,
     dropout=0.0, act=nn.ReLU(), graph_linear_num_layers=4)# clf_model.apply(init_weights)
    

In [9]:
if CUDA:
    GNN_cVAE.cuda()

In [11]:
GNN_cVAE.load_state_dict(torch.load('./save_model/tg_GNN_cVAE_2021-09-28_08-22.pt'))
GNN_cVAE.eval()

GraphcVAE(
  (encoder): Encoder(
    (GraphEncoding): GraphSAGE(17338, 256, num_layers=1)
    (LinearEncoding): ModuleList(
      (0): Linear(256, 256, bias=True)
      (1): Linear(256, 256, bias=True)
      (2): Linear(256, 256, bias=True)
      (3): Linear(256, 256, bias=True)
      (4): Linear(256, 128, bias=True)
    )
    (activation): ELU(alpha=1.0)
    (TotalLinearEncoding): ModuleList(
      (0): Linear(129, 128, bias=True)
      (1): Linear(128, 128, bias=True)
      (2): Linear(128, 64, bias=True)
    )
  )
  (decoder): Decoder(
    (z_layers): ModuleList(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): Linear(in_features=32, out_features=32, bias=True)
    )
    (GraphEncoding): GraphSAGE(17338, 256, num_layers=1)
    (clf_layers): ModuleList(
      (0): Linear(288, 256, bias=True)
      (1): Linear(256, 256, bias=True)
      (2): Linear(256, 256, bias=True)
      (3): Linear(256, 256, bias=True)
      (4): Linear(256, 1, bias=True)
    )
    (sigmoi

In [38]:
z = torch.randn(32)
z

tensor([ 1.0953,  0.0853,  0.3722,  0.7187,  1.1736,  0.5100, -0.0611,  0.7975,
         0.3343,  0.3685, -0.0359,  0.7883,  1.7624,  0.1117,  0.7646,  0.0204,
        -0.7119, -0.5214,  1.0306, -1.5767, -0.1559, -0.7975, -1.1704, -1.8236,
        -0.0195,  1.1334, -1.7669, -2.5623, -0.2870,  0.9759,  0.9001, -1.2198])

In [43]:
z = z.cuda()
for _ in range(10):
    index = random.randint(0,len(train_set))
    print(index)
    data = train_set[index]
    data = data.cuda()
    l = GNN_cVAE.decoder(data.x, data.edge_index, z)
    print(l.view(-1))
    print(data.y)
    print()

289
tensor([0.2689, 0.2689, 0.2689, 0.2689, 0.2689, 0.2689, 0.2689, 0.2689, 0.2689,
        0.2689, 0.2689, 0.2689, 0.2689, 0.2689, 0.2689, 0.2689, 0.2689, 0.2689,
        0.2689, 0.2689, 0.2689, 0.2689, 0.2689, 0.2689, 0.2689],
       device='cuda:0', grad_fn=<ViewBackward>)
tensor([1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1.], device='cuda:0')

982
tensor([0.2689, 0.2689, 0.2689, 0.3276, 0.2689, 0.2689, 0.2689, 0.9817, 0.2689,
        0.2730, 0.2689, 0.2689, 0.2689, 0.2689, 0.2689, 0.2689, 0.2689, 0.2689,
        0.2689, 0.2689, 0.2689, 0.2689, 0.2689, 0.2689, 0.9959],
       device='cuda:0', grad_fn=<ViewBackward>)
tensor([0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1.], device='cuda:0')

1214
tensor([0.2689, 0.2689, 0.2689, 0.2689, 0.2689, 0.2689, 0.2689, 0.2689, 0.2695,
        0.2689, 0.2689, 0.2689, 0.9952, 0.2689, 0.2689, 0.2689, 0.9960, 0.2689,
        0.2